In [ ]:
# This tests the methodology of reading in a csv file of companion detection data
# and produces a 1D contrast curve

# Created 2019 May 6 by E.S.

In [1]:
import numpy as np
import pandas as pd

In [2]:
file_name = "test_csv_file.csv"

In [3]:
info_file = pd.read_csv(file_name)

In [ ]:
# For groups of rows defined by 
#      A.) a common value of rad_asec
#      B.) a common value of ampl_linear_norm
# 1. find median value of S/N for each group
# 2. select lowest value of ampl_linear_norm which provides a minimum X S/N

In [14]:
# group by radius and ampl_linear_norm

info_file_grouped_rad_ampl = info_file.groupby(["rad_asec", "ampl_linear_norm"], 
                                            axis=0, 
                                            as_index=False).median()

In [53]:
# for each radius, find ampl_linear_norm with S/N > threshold_s2n

threshold_s2n = 2

# unique radius values
unique_rad_vals = info_file_grouped_rad_ampl["rad_asec"].unique()

# initialize array for contrast curve
contrast_curve = {"rad_asec": np.nan*np.ones(len(unique_rad_vals)),
                  "ampl_linear_norm": np.nan*np.ones(len(unique_rad_vals))}
contrast_curve_pd = pd.DataFrame(contrast_curve)

print(contrast_curve_pd)

# loop over unique radius values
for t in range(0,len(unique_rad_vals)):
    
    # subset of data with the right radius from the host star
    data_right_rad = info_file_grouped_rad_ampl.where(\
                                                        info_file_grouped_rad_ampl["rad_asec"] == unique_rad_vals[t]\
                                                       )
    # sub-subset of data with at least the minimum S/N
    data_right_s2n_presort = data_right_rad.where(\
                                         data_right_rad["s2n"] >= threshold_s2n\
                                         ).dropna()
    
    # the row of data with the minimum S/N above the minimum threshold    
    data_right_s2n_postsort = data_right_s2n_presort.where(\
                                                           data_right_s2n_presort["s2n"] == data_right_s2n_presort["s2n"].min()\
                                                          ).dropna()
    '''
    print(data_right_s2n_postsort)
    print(data_right_s2n_postsort["rad_asec"].values[0])
    print(data_right_s2n_postsort["ampl_linear_norm"].values[0])
    '''
    
    # append companion radius and amplitude values
    contrast_curve_pd.at[t,"rad_asec"] = data_right_s2n_postsort["rad_asec"].values[0]
    contrast_curve_pd.at[t,"ampl_linear_norm"] = data_right_s2n_postsort["ampl_linear_norm"].values[0]
    
    '''
    print("------------")
    print(data_right_rad)
    print("-")
    print(data_right_s2n_presort)
    print("-")
    print(data_right_s2n_postsort)
    '''

   ampl_linear_norm  rad_asec
0               NaN       NaN
1               NaN       NaN
   rad_asec  ampl_linear_norm  Unnamed: 0  angle_deg        noise      s2n  \
0       0.3               0.9         0.0       60.0  7041.697528  3.18424   

         signal  
0  21262.218055  
0.3
0.9
------------
   rad_asec  ampl_linear_norm  Unnamed: 0  angle_deg        noise       s2n  \
0       0.3               0.9         0.0       60.0  7041.697528  3.184240   
1       0.3               1.0         0.0       60.0  7295.982475  3.238479   
2       NaN               NaN         NaN        NaN          NaN       NaN   
3       NaN               NaN         NaN        NaN          NaN       NaN   

         signal  
0  21262.218055  
1  23577.016784  
2           NaN  
3           NaN  
-
   rad_asec  ampl_linear_norm  Unnamed: 0  angle_deg        noise       s2n  \
0       0.3               0.9         0.0       60.0  7041.697528  3.184240   
1       0.3               1.0         0.0       60

In [57]:
# write out to csv
contrast_curve_pd.to_csv("contrast_test.csv", sep = ",", columns = ["rad_asec","ampl_linear_norm"])